In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

sys.path.append('../')
from src.util.functions import Functions as F

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = '../data'
path_csv = f'{path_data}/csv'

csv_out_name = f'{path_csv}/covid19-{level}.csv'
pop_regions_name = f'{path_csv}/popolazione-regioni.csv'
icu_regions_name = f'{path_csv}/terapie-intensive.csv'
regions_name = f'{path_csv}/regioni.csv'

In [2]:
url_cum = F.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,note,ingressi_terapia_intensiva,note_test,note_casi,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7618,2021-02-20T17:00:00,ITA,19,Sicilia,38.115697,13.362357,862,145,1007,28899,...,NaN,3.0,NaN,NaN,148579.0,0.0,1681846.0,570421.0,ITG,ITG1
7619,2021-02-20T17:00:00,ITA,9,Toscana,43.769231,11.255889,746,147,893,12844,...,NaN,4.0,NaN,NaN,147411.0,442.0,2358166.0,210420.0,ITI,ITI1
7620,2021-02-20T17:00:00,ITA,10,Umbria,43.106758,12.388247,459,86,545,7975,...,"Si fa presente che 16 dei ricoveri NON UTI, so...",9.0,NaN,NaN,42722.0,0.0,668448.0,100126.0,ITI,ITI2
7621,2021-02-20T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,8,1,9,125,...,NaN,0.0,NaN,NaN,7959.0,0.0,72686.0,2721.0,ITC,ITC2


In [4]:
list(df_raw_r.columns)

['data',
 'stato',
 'codice_regione',
 'denominazione_regione',
 'lat',
 'long',
 'ricoverati_con_sintomi',
 'terapia_intensiva',
 'totale_ospedalizzati',
 'isolamento_domiciliare',
 'totale_positivi',
 'variazione_totale_positivi',
 'nuovi_positivi',
 'dimessi_guariti',
 'deceduti',
 'casi_da_sospetto_diagnostico',
 'casi_da_screening',
 'totale_casi',
 'tamponi',
 'casi_testati',
 'note',
 'ingressi_terapia_intensiva',
 'note_test',
 'note_casi',
 'totale_positivi_test_molecolare',
 'totale_positivi_test_antigenico_rapido',
 'tamponi_test_molecolare',
 'tamponi_test_antigenico_rapido',
 'codice_nuts_1',
 'codice_nuts_2']

In [5]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'casi_da_sospetto_diagnostico', 'casi_da_screening', 'note', 'note_test', 'note_casi']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,...,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,...,0,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,...,18,148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7618,2021-02-20T17:00:00,19,Sicilia,862,145,1007,28899,29906,-1663,474,...,148579,2252267,1090761.0,3.0,148579.0,0.0,1681846.0,570421.0,ITG,ITG1
7619,2021-02-20T17:00:00,9,Toscana,746,147,893,12844,13737,527,953,...,147853,2568586,1359653.0,4.0,147411.0,442.0,2358166.0,210420.0,ITI,ITI1
7620,2021-02-20T17:00:00,10,Umbria,459,86,545,7975,8520,81,363,...,42722,768574,307175.0,9.0,42722.0,0.0,668448.0,100126.0,ITI,ITI2
7621,2021-02-20T17:00:00,2,Valle d'Aosta,8,1,9,125,134,1,4,...,7959,75407,43273.0,0.0,7959.0,0.0,72686.0,2721.0,ITC,ITC2


In [6]:
# Compute most recent couple of dates
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = F.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2021-02-20', '2021-02-19')

In [7]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2021-02-20T17:00:00,1,Piemonte,1857,130,1987,10621,12608,171,765,...,238147,2423607,1201114.0,5.0,234093.0,4054.0,2013821.0,409786.0,ITC,ITC1
1,2021-02-20T17:00:00,2,Valle d'Aosta,8,1,9,125,134,1,4,...,7959,75407,43273.0,0.0,7959.0,0.0,72686.0,2721.0,ITC,ITC2
2,2021-02-20T17:00:00,3,Lombardia,3722,382,4104,49261,53365,1576,3019,...,576766,6331879,3021909.0,31.0,571580.0,5186.0,6014678.0,317201.0,ITC,ITC4
3,2021-02-20T17:00:00,5,Veneto,772,100,872,20915,21787,300,1244,...,325789,4760865,1372199.0,8.0,323036.0,2753.0,3947066.0,813799.0,ITH,ITH3
4,2021-02-20T17:00:00,6,Friuli Venezia Giulia,348,60,408,8844,9252,-109,326,...,73679,1291733,462473.0,6.0,69463.0,4216.0,1214064.0,77669.0,ITH,ITH4
5,2021-02-20T17:00:00,7,Liguria,509,53,562,4275,4837,5,361,...,75508,983717,420762.0,2.0,75508.0,0.0,896513.0,87204.0,ITC,ITC3
6,2021-02-20T17:00:00,8,Emilia-Romagna,1899,176,2075,33058,35133,196,1724,...,244170,3603939,1454701.0,11.0,244079.0,91.0,3271027.0,332912.0,ITH,ITH5
7,2021-02-20T17:00:00,9,Toscana,746,147,893,12844,13737,527,953,...,147853,2568586,1359653.0,4.0,147411.0,442.0,2358166.0,210420.0,ITI,ITI1
8,2021-02-20T17:00:00,10,Umbria,459,86,545,7975,8520,81,363,...,42722,768574,307175.0,9.0,42722.0,0.0,668448.0,100126.0,ITI,ITI2
9,2021-02-20T17:00:00,11,Marche,532,80,612,8081,8693,192,547,...,63251,787557,476674.0,3.0,63251.0,0.0,736191.0,51366.0,ITI,ITI3


In [8]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2021-02-19T17:00:00,1,Piemonte,1849,130,1979,10458,12437,431,1307,...,237382,2403012,1195422.0,10.0,233436.0,3946.0,2007364.0,395648.0,ITC,ITC1
1,2021-02-19T17:00:00,2,Valle d'Aosta,9,2,11,122,133,-3,8,...,7955,75155,43154.0,0.0,7955.0,0.0,72523.0,2632.0,ITC,ITC2
2,2021-02-19T17:00:00,3,Lombardia,3733,373,4106,47683,51789,1816,3724,...,573747,6287867,3007961.0,22.0,568878.0,4869.0,5982546.0,305321.0,ITC,ITC4
3,2021-02-19T17:00:00,5,Veneto,790,98,888,20599,21487,-499,657,...,324545,4716720,1368822.0,6.0,321792.0,2753.0,3941416.0,775304.0,ITH,ITH3
4,2021-02-19T17:00:00,6,Friuli Venezia Giulia,349,56,405,8956,9361,-49,310,...,73353,1281941,459555.0,3.0,69247.0,4106.0,1208281.0,73660.0,ITH,ITH4
5,2021-02-19T17:00:00,7,Liguria,523,58,581,4251,4832,22,274,...,75147,976976,418181.0,2.0,75147.0,0.0,892532.0,84444.0,ITC,ITC3
6,2021-02-19T17:00:00,8,Emilia-Romagna,1902,183,2085,32852,34937,8,1821,...,242462,3571996,1448201.0,20.0,242378.0,84.0,3254146.0,317850.0,ITH,ITH5
7,2021-02-19T17:00:00,9,Toscana,723,149,872,12338,13210,407,924,...,146900,2547014,1348205.0,14.0,146482.0,418.0,2344695.0,202319.0,ITI,ITI1
8,2021-02-19T17:00:00,10,Umbria,462,83,545,7894,8439,21,301,...,42359,763845,306231.0,7.0,42359.0,0.0,665142.0,98703.0,ITI,ITI2
9,2021-02-19T17:00:00,11,Marche,524,82,606,7895,8501,145,488,...,62704,781587,472892.0,7.0,62704.0,0.0,731702.0,49885.0,ITI,ITI3


In [9]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = df_raw_r_1
# df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
# df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

In [10]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = df_raw_r_0
# df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
# df_r_0.sort_values(by='codice_regione').reset_index()

In [11]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione
0,2021-02-20T17:00:00,1,Piemonte,1857,130,1987,10621,12608,171,765,...,2423607,1201114.0,5.0,234093.0,4054.0,2013821.0,409786.0,ITC,ITC1,4.341375e+06
1,2021-02-20T17:00:00,2,Valle d'Aosta,8,1,9,125,134,1,4,...,75407,43273.0,0.0,7959.0,0.0,72686.0,2721.0,ITC,ITC2,1.255010e+05
2,2021-02-20T17:00:00,3,Lombardia,3722,382,4104,49261,53365,1576,3019,...,6331879,3021909.0,31.0,571580.0,5186.0,6014678.0,317201.0,ITC,ITC4,1.010397e+07
3,2021-02-20T17:00:00,5,Veneto,772,100,872,20915,21787,300,1244,...,4760865,1372199.0,8.0,323036.0,2753.0,3947066.0,813799.0,ITH,ITH3,4.907704e+06
4,2021-02-20T17:00:00,6,Friuli Venezia Giulia,348,60,408,8844,9252,-109,326,...,1291733,462473.0,6.0,69463.0,4216.0,1214064.0,77669.0,ITH,ITH4,1.211357e+06
5,2021-02-20T17:00:00,7,Liguria,509,53,562,4275,4837,5,361,...,983717,420762.0,2.0,75508.0,0.0,896513.0,87204.0,ITC,ITC3,1.543127e+06
6,2021-02-20T17:00:00,8,Emilia-Romagna,1899,176,2075,33058,35133,196,1724,...,3603939,1454701.0,11.0,244079.0,91.0,3271027.0,332912.0,ITH,ITH5,4.467118e+06
7,2021-02-20T17:00:00,9,Toscana,746,147,893,12844,13737,527,953,...,2568586,1359653.0,4.0,147411.0,442.0,2358166.0,210420.0,ITI,ITI1,3.722729e+06
8,2021-02-20T17:00:00,10,Umbria,459,86,545,7975,8520,81,363,...,768574,307175.0,9.0,42722.0,0.0,668448.0,100126.0,ITI,ITI2,8.802850e+05
9,2021-02-20T17:00:00,11,Marche,532,80,612,8081,8693,192,547,...,787557,476674.0,3.0,63251.0,0.0,736191.0,51366.0,ITI,ITI3,1.518400e+06


In [12]:
# Update icu csv data
df_icu_updated_r = pd.DataFrame.from_dict(F.icu_data())
df_icu_updated_r.to_csv(icu_regions_name, index=False)

In [13]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, left_on='denominazione_regione', right_on='regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione,regione,ricoverati_area_non_critica,posti_letto_area_non_critica,ricoverati_terapia_intensiva,posti_letto_terapia_intensiva,posti_letto_terapia_intensiva_attivabili
0,2021-02-20T17:00:00,1,Piemonte,1857,130,1987,10621,12608,171,765,...,409786.0,ITC,ITC1,4.341375e+06,Piemonte,1849,5824,130,628,99
1,2021-02-20T17:00:00,2,Valle d'Aosta,8,1,9,125,134,1,4,...,2721.0,ITC,ITC2,1.255010e+05,Valle d'Aosta,9,227,2,20,12
2,2021-02-20T17:00:00,3,Lombardia,3722,382,4104,49261,53365,1576,3019,...,317201.0,ITC,ITC4,1.010397e+07,Lombardia,3733,10290,373,1248,282
3,2021-02-20T17:00:00,5,Veneto,772,100,872,20915,21787,300,1244,...,813799.0,ITH,ITH3,4.907704e+06,Veneto,790,6000,98,1000,0
4,2021-02-20T17:00:00,6,Friuli Venezia Giulia,348,60,408,8844,9252,-109,326,...,77669.0,ITH,ITH4,1.211357e+06,Friuli Venezia Giulia,349,1277,56,175,0
5,2021-02-20T17:00:00,7,Liguria,509,53,562,4275,4837,5,361,...,87204.0,ITC,ITC3,1.543127e+06,Liguria,523,1775,58,216,18
6,2021-02-20T17:00:00,8,Emilia-Romagna,1899,176,2075,33058,35133,196,1724,...,332912.0,ITH,ITH5,4.467118e+06,Emilia-Romagna,1902,6126,183,757,0
7,2021-02-20T17:00:00,9,Toscana,746,147,893,12844,13737,527,953,...,210420.0,ITI,ITI1,3.722729e+06,Toscana,723,5033,149,567,16
8,2021-02-20T17:00:00,10,Umbria,459,86,545,7975,8520,81,363,...,100126.0,ITI,ITI2,8.802850e+05,Umbria,462,851,83,141,2
9,2021-02-20T17:00:00,11,Marche,532,80,612,8081,8693,192,547,...,51366.0,ITI,ITI3,1.518400e+06,Marche,524,1176,82,233,40


In [14]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [15]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['posti_letto_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['posti_letto_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2021-02-20T17:00:00,3,Lombardia,3722,382,4104,49261,53365,1576,3019,...,30.61,1578,-705,44012,13948.0,37,1406,6.86,0.02988,5.71
1,2021-02-20T17:00:00,8,Emilia-Romagna,1899,176,2075,33058,35133,196,1724,...,23.25,206,-97,31943,6500.0,23,1489,5.40,0.03859,5.47
2,2021-02-20T17:00:00,15,Campania,1289,112,1401,68951,70352,1123,1677,...,18.06,1145,61,20893,14181.0,11,543,8.03,0.02898,4.35
3,2021-02-20T17:00:00,5,Veneto,772,100,872,20915,21787,300,1244,...,10.00,316,587,44145,3377.0,18,926,2.82,0.02535,6.64
4,2021-02-20T17:00:00,9,Toscana,746,147,893,12844,13737,527,953,...,25.93,506,29,21572,11448.0,15,411,4.42,0.02560,3.97
5,2021-02-20T17:00:00,12,Lazio,1883,228,2111,33335,35446,-162,921,...,24.18,-117,-69,31269,17792.0,32,1051,2.95,0.01570,3.83
6,2021-02-20T17:00:00,16,Puglia,1251,164,1415,31669,33084,-246,905,...,28.82,-237,31,9880,3852.0,23,1128,9.16,0.02258,3.49
7,2021-02-20T17:00:00,1,Piemonte,1857,130,1987,10621,12608,171,765,...,20.70,163,-542,20595,5692.0,12,582,3.71,0.01762,5.49
8,2021-02-20T17:00:00,11,Marche,532,80,612,8081,8693,192,547,...,34.33,186,59,5970,3782.0,8,347,9.16,0.03602,4.17
9,2021-02-20T17:00:00,21,P.A. Bolzano,249,36,285,6846,7131,-147,477,...,36.00,-142,-47,11195,660.0,4,620,4.26,89.55325,9626.88


In [16]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [17]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'posti_letto_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,posti_letto_terapia_intensiva,saturazione_terapia_intensiva
0,Umbria,86,55,141,60.99
1,Abruzzo,75,126,201,37.31
2,P.A. Bolzano,36,64,100,36.00
3,Marche,80,153,233,34.33
4,Friuli Venezia Giulia,60,115,175,34.29
5,Molise,13,26,39,33.33
6,Lombardia,382,866,1248,30.61
7,P.A. Trento,27,63,90,30.00
8,Puglia,164,405,569,28.82
9,Toscana,147,420,567,25.93


In [18]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [19]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../data/csv/regioni/covid19-veneto.csv
Aggiornato ../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../data/csv/regioni/covid19-liguria.csv
Aggiornato ../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../data/csv/regioni/covid19-toscana.csv
Aggiornato ../data/csv/regioni/covid19-umbria.csv
Aggiornato ../data/csv/regioni/covid19-marche.csv
Aggiornato ../data/csv/regioni/covid19-lazio.csv
Aggiornato ../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../data/csv/regioni/covid19-molise.csv
Aggiornato ../data/csv/regioni/covid19-campania.csv
Aggiornato ../data/csv/regioni/covid19-puglia.csv
Aggiornato ../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../data/csv/regioni/covid19-calabria.csv
Aggiornato ../data/csv/regioni/covid19-sicilia.csv
Aggiornato ../data/csv/regioni/covid19-sardegna.csv
Ag

In [20]:
# df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
# df_tb

In [21]:
# Initialize df_taa
# df_taa = Functions.replace_bt_with_taa(df_tb[:2])
# for i in range(2, df_tb.shape[0], 2):
#     df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
# df_taa = add_cols(df_taa)
# df_taa

In [22]:
# region_name = df_taa.denominazione_regione.iloc[0].lower()
# csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
# df_taa.to_csv(csv_out_region_name, index=False)
# print('Aggiornato {file}'.format(file=csv_out_region_name))